In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
root_dir = "../input/screw-dataset/"
train_csv_filepath = root_dir + "train.csv"

train_df = pd.read_csv(train_csv_filepath)

In [3]:
resize_w = 256
resize_h = 256
channel = 3

import cv2
def resize(tmp_image):
    return cv2.resize(tmp_image , (resize_h, resize_w))
def to_4d(tmp_image):
    return tmp_image.reshape(1, resize_h, resize_w, channel)
def normalize(tmp_image):
    return tmp_image / 255.0

def load_preprocessed_image(image_filepath):
    tmp_image = cv2.imread(image_filepath)
    tmp_image = resize(tmp_image)
    tmp_image = normalize(tmp_image)
    tmp_image = to_4d(tmp_image)

    return tmp_image

In [4]:
import numpy as np
from keras.utils import np_utils

images = None
for fn in train_df['filename']:
    image_filepath = root_dir + 'train/' + fn
    tmp_image = load_preprocessed_image(image_filepath)
    if (images is None):
        images = tmp_image
    else:
        images = np.vstack((images, tmp_image))

anomaly_flags = np.array([flag for flag in train_df['anomaly']])
anomaly_flags = np_utils.to_categorical(anomaly_flags, 2)

In [5]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.333, random_state=0)

for train_index, test_index in sss.split(images, anomaly_flags):
    X_train = images[train_index]
    y_train = anomaly_flags[train_index]
    X_test = images[test_index]
    y_test = anomaly_flags[test_index]

In [6]:

tmp = pd.DataFrame(y_train[:, 1]).value_counts().values
print(tmp)
label_ok_num = tmp[0]
label_ng_num = tmp[1]

while(label_ok_num != label_ng_num):
    rand_index = np.random.randint(0, len(y_train))

    label_is_ng = (y_train[rand_index, 1] == 1.0)
    if label_is_ng:
        X_train = np.vstack((X_train, [X_train[rand_index]]))
        y_train = np.vstack((y_train, [y_train[rand_index]]))
        label_ng_num += 1
    print(label_ng_num, end='\r')

[167  33]


In [7]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rotation_range=360)

datagen.fit(X_train)

In [9]:
import numpy as np


from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam

def cnn_model():
    model = Sequential()

    model.add(Conv2D(filters=10, kernel_size=(4,4), padding='same', input_shape=(256, 256, 3), activation='relu'))
    model.add(Conv2D(filters=10, kernel_size=(3,3), padding='same', input_shape=(64, 64, 8), activation='relu'))
    model.add(Conv2D(filters=10, kernel_size=(2,2), padding='same', input_shape=(16, 16, 16), activation='relu'))
    model.add(Conv2D(filters=10, kernel_size=(2,2), padding='same', input_shape=(8, 8, 16), activation='relu'))

    model.add(Flatten())
    model.add(Dense(2, activation='softmax'))
    adam = Adam(lr=1e-4, decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [10]:
epochs = 500
n_batch = 8

model = cnn_model()
model.fit_generator(datagen.flow(X_train, y_train, batch_size=n_batch),
                    steps_per_epoch=len(X_train) / n_batch,
                    epochs=epochs,
                    validation_data=(X_test, y_test))


/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/500
41/41 [==============================] - 7s 139ms/step - loss: 0.7302 - accuracy: 0.4064 - val_loss: 0.7275 - val_accuracy: 0.1900
Epoch 2/500
41/41 [==============================] - 5s 115ms/step - loss: 0.6943 - accuracy: 0.4967 - val_loss: 0.6899 - val_accuracy: 0.7100
Epoch 3/500
41/41 [==============================] - 5s 120ms/step - loss: 0.6942 - accuracy: 0.4873 - val_loss: 0.6987 - val_accuracy: 0.2200
Epoch 4/500
41/41 [==============================] - 5s 114ms/step - loss: 0.6944 - accuracy: 0.4278 - val_loss: 0.6873 - val_accuracy: 0.7400
Epoch 5/500
41/41 [==============================] - 6s 134ms/step - loss: 0.6933 - accuracy: 0.5112 - val_loss: 0.6965 - val_accuracy: 0.4100
Epoch 6/500
41/41 [==============================] - 5s 116ms/step - loss: 0.6936 - accuracy: 0.4947 - val_loss: 0.6978 - val_accuracy: 0.4300
Epoch 7/500
41/41 [==============================] - 5s 119ms/step - loss: 0.6928 - accuracy: 0.5382 - val_loss: 0.6960 - val_accuracy: 0.4000

NameError: name 'model' is not defined